In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import brown
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
from nltk.corpus import words
import seaborn as sns

In [38]:
### INITIALIZING BLOCK OF CODE - LOADING UP THE ESSENTIAL LEXICONS AND DATA FILES AND LISTS ###

# review data files

hp = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/basic_dataHP.csv')
le = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/basic_dataLEN22.csv')
de = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/basic_DELL_data.csv')
ma = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/basic_MAC_data.csv')

# gold label files

Dlab = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/GOLD_DELL_LABELS.csv')
Hlab = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/GOLD_HP_LABELS.csv')
Llab = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/GOLD_LABELS_len.csv')
Mlab = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/disser/GOLD_MAC_LABELS.csv')

# manually selected topic words

per = pd.read_csv('/Users/Akashgupta/Desktop/data_files/per.csv')
cus = pd.read_csv('/Users/Akashgupta/Desktop/data_files/cus.csv')
pri = pd.read_csv('/Users/Akashgupta/Desktop/data_files/pri.csv')
per = list(per['per'])
cus = list(cus['cus'])
pri = list(pri['pri'])

# pure lexicons

BN = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/files_post/BP.csv')
BP = pd.read_csv('/Users/Akashgupta/Desktop/PRO-Desktop/NEWERFIN/files_post/BPP.csv')

# misspelled positive and negative words

POS = pd.read_csv('/Users/Akashgupta/Desktop/data_files/positives.csv')
NEG = pd.read_csv('/Users/Akashgupta/Desktop/data_files/negatives.csv')

# list of negation words

negation_words = ["no", "not", "don't", "without", "didn't", "never", "won't", 'wont']

In [39]:
HP = hp.rename(columns={'HP': 'review'})
HP['brand'] = ['HP' for i in range(len(HP['review']))]
HP['label'] = Hlab['LABELS']

LE = le.rename(columns={'LENOVO': 'review'})
LE['brand'] = ['LENOVO' for i in range(len(LE['review']))]
LE['label'] = Llab['LABELS']

DE = de.rename(columns={'DELL': 'review'})
DE['brand'] = ['DELL' for i in range(len(DE['review']))]
DE['label'] = Dlab['LABELS']

MA = ma.rename(columns={'Macbook': 'review'})
MA['brand'] = ['MAC' for i in range(len(MA['review']))]
MA['label'] = Mlab['LABELS']

df = pd.concat([HP, LE, DE, MA]).reset_index(drop=True, inplace=False)

In [40]:
df

,review,brand,label
0,"Very slim ,light weight easily carry in travel...",HP,1
1,I bought this exchanging my Acer Aspire V5 int...,HP,1
2,initial impression is excellent...light weight...,HP,1
3,I am writing this review after using the lapto...,HP,1
4,Don't buy these product as it is total cheap q...,HP,0
...,...,...,...
1992,"Nice but it's old now, go for 8th generation",MAC,0
1993,Sir that have battery draining problem so I'll...,MAC,0
1994,Smooth like airstrikes.,MAC,1
1995,It’s been a month since I bought this and I ca...,MAC,1


In [41]:
sents = []
for text in df['review']:
    sents += nltk.sent_tokenize(text)

In [42]:
sents2 = []
for text in df['review']:
    sents2 += re.split(r'\.|\n|and|but|And|But|&', text)

sents3 = []
for ss in sents2:
    if len(nltk.word_tokenize(ss)) > 1:
        sents3.append(ss)

In [43]:
p2 = r"\w+(?:[-']\w+)*"

tokenized = []
tagtoken = []
tdf = pd.DataFrame()
for i in range(len(sents)):
    c = [w for w in nltk.regexp_tokenize(sents[i], p2) if len(w)>1]
    a = nltk.pos_tag(c)
    b = [(w, t) for (w, t) in a if t not in ['DT', 'IN', 'EX', 'CC', 'CD']]
    tokenized.append(c)
    tagtoken.append(b)
tdf['toktext'] = tokenized
tdf['tagtext'] = tagtoken

In [9]:
tdf

,toktext,tagtext
0,"[Very, slim, light, weight, easily, carry, in,...","[(Very, RB), (slim, JJ), (light, NN), (weight,..."
1,"[Good, battery, time, Min, 5hrs, For, office, ...","[(Good, JJ), (battery, NN), (time, NN), (Min, ..."
2,"[bought, this, exchanging, my, Acer, Aspire, V...","[(bought, VBD), (exchanging, VBG), (my, PRP$),..."
3,"[There, is, absolutely, no, heating, or, any, ...","[(is, VBZ), (absolutely, RB), (heating, NN), (..."
4,"[This, fits, my, requirement, of, day, to, day...","[(fits, VBZ), (my, PRP$), (requirement, NN), (..."
...,...,...
6428,"[But, not, want, that, plz, give, me, back]","[(not, RB), (want, VB), (plz, NN), (give, VB),..."
6429,"[Smooth, like, airstrikes]","[(Smooth, NNP), (airstrikes, NNS)]"
6430,"[It, been, month, since, bought, this, and, ca...","[(It, PRP), (been, VBN), (month, NN), (bought,..."
6431,"[Easy, to, use, and, battery, life, is, amazing]","[(Easy, NNP), (to, TO), (use, VB), (battery, V..."


In [44]:
vocab = []
taggedvocab = []
for i in range(len(tdf['tagtext'])):
    for j in range(len(tdf['tagtext'][i])):
        tdf['tagtext'][i][j] = list(tdf['tagtext'][i][j])
        tdf['tagtext'][i][j][0] = tdf['tagtext'][i][j][0].lower()
        vocab.append(tdf['tagtext'][i][j][0])
        taggedvocab.append((tdf['tagtext'][i][j][0], tdf['tagtext'][i][j][1]))

In [45]:
V = list(set(vocab))

In [12]:
len(taggedvocab)

58780

In [46]:
data = dict(nltk.ConditionalFreqDist((word.lower(), tag) for (word, tag) in taggedvocab))

In [47]:
DD = dict(nltk.FreqDist([word for word in vocab if len(word)>4]))
S_dict = sorted(DD.items(), key=lambda item: item[1], reverse=True)

In [15]:
S_dict[:30]

[('laptop', 1308),
 ('product', 674),
 ('battery', 519),
 ('performance', 337),
 ('quality', 325),
 ('apple', 276),
 ('price', 274),
 ('amazon', 243),
 ('money', 224),
 ('which', 223),
 ('display', 214),
 ('screen', 213),
 ('macbook', 212),
 ('great', 201),
 ('light', 200),
 ('working', 190),
 ('windows', 188),
 ("don't", 174),
 ('office', 173),
 ('bought', 170),
 ('using', 167),
 ('really', 166),
 ('better', 159),
 ('gaming', 158),
 ('service', 144),
 ('keyboard', 135),
 ('other', 134),
 ('weight', 131),
 ('awesome', 131),
 ('issue', 131)]

In [48]:
g_map = {}
for k in data.keys():
    g_map[k] = data[k].max()

In [49]:
g_map['laptop'] = 'NN'

In [50]:
named_entity = []
action_words = []
descriptive_words = []

for key in g_map.keys():
    if g_map[key] in ['NN', 'NNP', 'NNS']:
        named_entity.append(key)
    elif g_map[key] in ['JJ', 'JJR', 'JJS']:
        descriptive_words.append(key)
    elif g_map[key] in ['VBD', 'VBG', 'VBZ', 'VB', 'VBN', 'VBP']:
        action_words.append(key)

In [51]:
STOPS = list(set(stopwords.words('english')) - set(negation_words))

In [52]:
regex = r'[a-zA-Z]{2,21}'
vect = CountVectorizer(stop_words='english')
DTM = vect.fit_transform(df['review'])
matrix = pd.DataFrame(DTM.toarray(), columns=vect.get_feature_names())

In [53]:
def cos_sim(a, b):
    cs = (np.dot(a, b))/(np.linalg.norm(a)*np.linalg.norm(b))
    return cs

In [54]:
# new named entity

ne_new = set(named_entity)-set(list(BP['word'])).intersection(set(named_entity))
ne_new2 = set(ne_new)-set(list(BN['word'])).intersection(set(ne_new))
ne_fin = list(ne_new2)

In [55]:
dw1_p = list(set(list(BP['word'])).intersection(set(named_entity)))
dw1_n = list(set(list(BN['word'])).intersection(set(named_entity)))

dw2_p = list(set(list(BP['word'])).intersection(set(descriptive_words)))
dw2_n = list(set(list(BN['word'])).intersection(set(descriptive_words)))

aw1_p = list(set(list(BP['word'])).intersection(set(action_words)))
aw1_n = list(set(list(BN['word'])).intersection(set(action_words)))

DW_p = dw1_p + dw2_p
DW_n = dw1_n + dw2_n

In [56]:
DW_p.remove('support')
DW_n.append('heating')

In [57]:
DW_n = DW_n + list(set(NEG))
DW_p = DW_p + list(set(POS))

In [79]:
DW_p.append('fast')

In [59]:
performance = []
customer = []
price = []

for w in per:
    if w not in ne_fin:
        performance.append(w)
    for w2 in ne_fin:
        try:
            dis = cos_sim(matrix[w], matrix[w2])
            if dis >= 0.30 and (w2 not in cus) and (w2 not in pri):
                performance.append(w2)
        except KeyError:
                pass
            
for w3 in cus:
    if w3 not in ne_fin:
        customer.append(w3)
    for w4 in ne_fin:
        try:
            dis = cos_sim(matrix[w3], matrix[w4])
            if dis >= 0.30 and (w4 not in performance) and (w4 not in pri):
                customer.append(w4)
        except KeyError:
                pass
            
for w5 in pri:
    if w5 not in ne_fin:
        price.append(w5)
    for w6 in ne_fin:
        try:
            dis = cos_sim(matrix[w5], matrix[w6])
            if dis >= 0.30 and (w6 not in performance) and (w6 not in customer):
                price.append(w6)
        except KeyError:
                pass

In [60]:
performance = list(set(performance))
customer = list(set(customer))
price = list(set(price))

In [61]:
price = ['pricing', 'range', 'value', 'price', 'sale', 'pay', 'money', 'cost', 'budget']

In [62]:
Uset = set(performance).union(set(customer))
U2set = set(Uset).union(set(price))

genset = list(set(ne_fin) - U2set)

In [80]:
per

['battery',
 'performance',
 'display',
 'screen',
 'light',
 'fast',
 'windows',
 'gaming',
 'slow',
 'keyboard',
 'weight',
 'design',
 'ssd',
 'backup',
 'sound',
 'processor',
 'ram',
 'build',
 'camera',
 'speed',
 'looks',
 'software',
 'heating',
 'charging',
 'graphics',
 'video',
 'body',
 'fan',
 'heavy',
 'usb',
 'intel',
 'games',
 'editing',
 'storage',
 'heat',
 'charge',
 'speakers',
 'plastic',
 'ports',
 'touch',
 'built',
 'charger',
 'browsing',
 'core',
 'videos',
 'boot',
 'lightweight',
 'card',
 'processing',
 'hardware',
 'cpu',
 'backlit',
 'memory']

In [32]:
# reformatting the performance words

for w7 in performance:
    for w8 in genset:
        try:
            dis = cos_sim(matrix[w7], matrix[w8])
            if dis >= 0.30 and len(w8)>5 and w8 not in performance:
                performance.append(w8)
        except KeyError:
                pass

In [33]:
# reformatting the performance words

performance = list(set(performance))

In [34]:
DW_n.append("disappointe")

In [35]:
performance.remove("disappointe")

In [36]:
performance.remove("laptop")
performance.remove("products")
performance.remove("student")

In [95]:
aw1_p

['breeze',
 'charming',
 'satisfies',
 'improve',
 'friendliness',
 'trusting',
 'exceeds',
 'upgraded',
 'protect',
 'master',
 'loving',
 'astonished',
 'enjoyed',
 'leads',
 'excited',
 'appreciated',
 'kudos',
 'integrated',
 'satisfy',
 'loves',
 'trust',
 'proves',
 'improved',
 'slick',
 'like',
 'promising',
 'restored',
 'exceeded',
 'gifted',
 'prefer',
 'works',
 'won',
 'love',
 'appreciate',
 'amaze',
 'accurate',
 'win',
 'helped',
 'pleasing',
 'entertaining',
 'improves',
 'supports',
 'afford',
 'dedicated',
 'loved',
 'assure',
 'likes',
 'delighted',
 'worked',
 'rectify',
 'convincing',
 'promised',
 'recommended',
 'zeal',
 'impressed',
 'enjoying',
 'supporting',
 'cherished',
 'recommend',
 'mesmerized',
 'amazing',
 'helping',
 'refunded',
 'enjoys',
 'amazed',
 'satisfied',
 'liked',
 'proving',
 'supported',
 'vouch',
 'trusted',
 'pleasant',
 'clean']

In [36]:
generals = ["product", "laptop", "office", "students", "overall", "machine", "student"]

In [37]:
neg_bigrams = []

for s in sents:
    b_list = list(ngrams([w.lower() for w in nltk.regexp_tokenize(s, p2) if len(w)>1
                          and w not in STOPS], 2))
    for bigram in b_list:
        if bigram[0] in negation_words:
            neg_bigrams.append(bigram)
        else:
            pass

In [38]:
pos_b = []
neg_b = []

for b in neg_bigrams:
    if (b[1] in DW_n) or (b[1] in aw1_n):
        pos_b.append(b)
    elif (b[1] in DW_p) or (b[1] in aw1_p):
        neg_b.append(b)

In [39]:
pairs = {}
for i in range(len(df['review'])):
    pairs[i] = [sent for sent in re.split(r'\.|\n|and|but|And|But', df['review'][i]) if len(sent) > 2]

In [40]:
positive_bigrams = list(set(pos_b))
negative_bigrams = list(set(neg_b))

In [41]:
p_nb_ex = []
n_nb_ex = []

for big in positive_bigrams:
    n_nb_ex.append(big[1])

for big2 in negative_bigrams:
    p_nb_ex.append(big2[1])

In [42]:
# descriptive lexicons - new

n_DW_n = list(set(DW_n) - set(n_nb_ex))             # NEGATIVE lexicon without neg bigram sentiment words
n_DW_p = list(set(DW_p) - set(p_nb_ex))             # POSITIVE lexicon without neg bigram sentiment words

# action lexicons - new
 
n_aw1_n = list(set(aw1_n) - set(n_nb_ex))           # NEGATIVE lexicon without neg bigram sentiment words
n_aw1_p = list(set(aw1_p) - set(p_nb_ex))           # POSITIVE lexicon without neg bigram sentiment words

In [43]:
n_sents = {}
pure_sents = {}

for k in pairs.keys():
    n_sents[k] = []
    pure_sents[k] = []

In [44]:
for k in pairs.keys():
    for se in pairs[k]:
        blist = list(ngrams([w.lower() for w in nltk.regexp_tokenize(se, p2) if len(w)>1
                              and w not in STOPS], 2))
        for bb in blist:
            if ((bb in negative_bigrams) or (bb in positive_bigrams)) and (se not in n_sents[k]):
                n_sents[k].append(se)
            else:
                pass
        
    for se2 in pairs[k]:
        if (se2 not in n_sents[k]) and (se2 not in pure_sents[k]):
            pure_sents[k].append(se2)

In [45]:
# all sentences

indexes = []
rev_sents = []

for k in pairs.keys():
    for S in pairs[k]:
        indexes.append(k)
        rev_sents.append(S)

i_data = pd.DataFrame()
i_data['indexes'] = indexes
i_data['sentences'] = rev_sents

In [46]:
indexes_negs = []
rev_sents_negs = []

for k in n_sents.keys():
    for S in n_sents[k]:
        indexes_negs.append(k)
        rev_sents_negs.append(S)

i3_data_negs = pd.DataFrame()
i3_data_negs['indexes'] = indexes_negs
i3_data_negs['sentences'] = rev_sents_negs

In [47]:
indexes_pure = []
rev_sents_pure = []

for k in pure_sents.keys():
    for S in pure_sents[k]:
        indexes_pure.append(k)
        rev_sents_pure.append(S)

i4_data_pure = pd.DataFrame()
i4_data_pure['indexes'] = indexes_pure
i4_data_pure['sentences'] = rev_sents_pure

In [48]:
n_score = []
p_d_score = []
p_a_score = []
length_stops = []
length_no_stops = []

for i in range(len(i_data['sentences'])):
    NS = 0
    PDS = 0
    PAS = 0
    
    tokens = [w.lower() for w in nltk.regexp_tokenize(i_data['sentences'][i], p2) if len(w)>1
              and w not in STOPS]
    tokens2 = [w.lower() for w in nltk.regexp_tokenize(i_data['sentences'][i], p2) if len(w)>1]
    b_tokens = list(ngrams(tokens, 2))
    
    if i_data['sentences'][i] in rev_sents_negs:
        for big in b_tokens:
            if big in positive_bigrams:
                NS += 2
            elif big in negative_bigrams:
                NS -= 2
            
        for word in tokens:
            if (word in n_DW_p) or (word in n_aw1_p):
                NS += 2
            elif (word in n_DW_n) or (word in n_aw1_n):
                NS -= 2
                    
    elif i_data['sentences'][i] in rev_sents_pure:
        for word in tokens:
            if (word in DW_p):
                PDS += 1
            elif (word in DW_n):
                PDS -= 1
                
        for word in tokens:
            if (word in aw1_p):
                PAS += 1
            elif (word in aw1_n):
                PAS -= 1
                
    n_score.append(NS)
    p_d_score.append(PDS)
    p_a_score.append(PAS)
    length_stops.append(len(tokens))
    length_no_stops.append(len(tokens2))

In [49]:
per_words = []
cus_words = []
pri_words = []
gen_words = []

for i in range(len(i_data['sentences'])):
    PW = 0
    CW = 0
    RW = 0
    GW = 0
    tokens = [w.lower() for w in nltk.regexp_tokenize(i_data['sentences'][i], p2) if len(w)>1
                  and w not in STOPS]

    for word in tokens:
        if word in performance:
            PW += 1
        elif word in customer:
            CW += 1
        elif word in price:
            RW += 1
        elif word in generals:
            GW += 1
        else:
            pass
    
    per_words.append(PW)
    cus_words.append(CW)
    pri_words.append(RW)
    gen_words.append(GW)

In [50]:
DF = pd.DataFrame()
DF['reviews'] = i_data['indexes']
DF['rev_sents'] = i_data['sentences']
DF['performance_words'] = per_words
DF['customer_words'] = cus_words
DF['price_words'] = pri_words
DF['general_words'] = gen_words
DF['negation_score'] = n_score
DF['descriptive_score'] = p_d_score
DF['action_score'] = p_a_score
DF['length_stops'] = length_stops
DF['length_no_stops'] = length_no_stops

In [51]:
DF

,reviews,rev_sents,performance_words,customer_words,price_words,general_words,negation_score,descriptive_score,action_score,length_stops,length_no_stops
0,0,"Very slim ,light weight easily carry in traveling",2,0,0,0,0,0,0,7,8
1,0,Good battery time,1,0,0,0,0,1,0,3,3
2,0,Min 5hrs,0,0,0,0,0,0,0,2,2
3,0,For office use best one,0,0,0,1,0,1,0,5,5
4,1,I bought this exchanging my Acer Aspire V5 int...,5,0,0,1,0,2,0,20,26
...,...,...,...,...,...,...,...,...,...,...,...
10705,1996,Battery is good,1,0,0,0,0,1,0,2,3
10706,1996,Sound is just awesome,1,0,0,0,0,1,0,2,4
10707,1996,not for gaming,1,0,0,0,0,0,0,2,3
10708,1996,price range is some high for me,0,0,2,0,0,0,0,3,7


In [52]:
DF.to_csv('/Users/Akashgupta/Desktop/data_files/16julR1.csv')

In [53]:
final = pd.read_csv('/Users/Akashgupta/Desktop/data_files/16julR1.csv')

In [54]:
fn = final.copy()

In [55]:
fn = final[['reviews', 'performance_words', 'customer_words', 'price_words', 'general_words',
            'negation_score', 'descriptive_score', 'action_score', 'length_stops', 'length_no_stops']]

In [56]:
FN = fn.copy()

In [57]:
FN['cumulative_score'] = FN['descriptive_score']+FN['action_score']+FN['negation_score']

In [58]:
FNN = FN.copy()
FNN = FN[['reviews', 'performance_words', 'customer_words', 'price_words', 'general_words', 'cumulative_score',
          'length_stops', 'length_no_stops']]

In [59]:
FNN.to_csv('/Users/Akashgupta/Desktop/data_files/16julR2.csv')

In [60]:
FNN

,reviews,performance_words,customer_words,price_words,general_words,cumulative_score,length_stops,length_no_stops
0,0,2,0,0,0,0,7,8
1,0,1,0,0,0,1,3,3
2,0,0,0,0,0,0,2,2
3,0,0,0,0,1,1,5,5
4,1,5,0,0,1,2,20,26
...,...,...,...,...,...,...,...,...
10705,1996,1,0,0,0,1,2,3
10706,1996,1,0,0,0,1,2,4
10707,1996,1,0,0,0,0,2,3
10708,1996,0,0,2,0,0,3,7


In [61]:
pmarks = []
cmarks = []
rmarks = []
gmarks = []

for i in range(len(FNN['reviews'])):
    if FNN['performance_words'][i] > 0:
        pmarks.append('P')
    else:
        pmarks.append('NP')

for i in range(len(FNN['reviews'])):
    if FNN['customer_words'][i] > 0:
        cmarks.append('C')
    else:
        cmarks.append('NC')
        
for i in range(len(FNN['reviews'])):
    if FNN['price_words'][i] > 0:
        rmarks.append('R')
    else:
        rmarks.append('NR')
        
for i in range(len(FNN['reviews'])):
    if FNN['general_words'][i] > 0:
        gmarks.append('G')
    else:
        gmarks.append('NG')

In [62]:
f3 = FNN.copy()
f3['p'] = pmarks
f3['c'] = cmarks
f3['r'] = rmarks
f3['g'] = gmarks

In [63]:
f3

,reviews,performance_words,customer_words,price_words,general_words,cumulative_score,length_stops,length_no_stops,p,c,r,g
0,0,2,0,0,0,0,7,8,P,NC,NR,NG
1,0,1,0,0,0,1,3,3,P,NC,NR,NG
2,0,0,0,0,0,0,2,2,NP,NC,NR,NG
3,0,0,0,0,1,1,5,5,NP,NC,NR,G
4,1,5,0,0,1,2,20,26,P,NC,NR,G
...,...,...,...,...,...,...,...,...,...,...,...,...
10705,1996,1,0,0,0,1,2,3,P,NC,NR,NG
10706,1996,1,0,0,0,1,2,4,P,NC,NR,NG
10707,1996,1,0,0,0,0,2,3,P,NC,NR,NG
10708,1996,0,0,2,0,0,3,7,NP,NC,R,NG


In [65]:
pscore = []
for i in range(len(f3['reviews'])):
    if f3['cumulative_score'][i]>0.0 and f3['p'][i]=='P':
        pscore.append(1)
    elif f3['cumulative_score'][i]<0.0 and f3['p'][i]=='P':
        pscore.append(-1)
    else:
        pscore.append(0)

In [68]:
cscore = []
for i in range(len(f3['reviews'])):
    if f3['cumulative_score'][i]>0.0 and f3['c'][i]=='C':
        cscore.append(1)
    elif f3['cumulative_score'][i]<0.0 and f3['c'][i]=='C':
        cscore.append(-1)
    else:
        cscore.append(0)

In [71]:
rscore = []
for i in range(len(f3['reviews'])):
    if f3['cumulative_score'][i]>0.0 and f3['r'][i]=='R':
        rscore.append(1)
    elif f3['cumulative_score'][i]<0.0 and f3['r'][i]=='R':
        rscore.append(-1)
    else:
        rscore.append(0)

In [74]:
gscore = []
for i in range(len(f3['reviews'])):
    if f3['cumulative_score'][i]>0.0 and f3['g'][i]=='G':
        gscore.append(1)
    elif f3['cumulative_score'][i]>0.0 and f3['p'][i]=='NP' and f3['c'][i]=='NC' and f3['r'][i]=='NR' and f3['g'][i]=='NG':
        gscore.append(1)
    elif f3['cumulative_score'][i]<0.0 and f3['g'][i]=='G':
        gscore.append(-1)
    elif f3['cumulative_score'][i]<0.0 and f3['p'][i]=='NP' and f3['c'][i]=='NC' and f3['r'][i]=='NR' and f3['g'][i]=='NG':
        gscore.append(-1)
    else:
        gscore.append(0)

In [77]:
f3.to_csv('/Users/Akashgupta/Desktop/data_files/16julR3.csv')

In [78]:
f22 = pd.DataFrame()
f22['reviews'] = f3['reviews']
f22['PS'] = pscore
f22['CS'] = cscore
f22['RS'] = rscore
f22['GS'] = gscore
f22['length_stops'] = f3['length_stops']
f22['length_no_stops'] = f3['length_no_stops']

In [79]:
f22

,reviews,PS,CS,RS,GS,length_stops,length_no_stops
0,0,0,0,0,0,7,8
1,0,1,0,0,0,3,3
2,0,0,0,0,0,2,2
3,0,0,0,0,1,5,5
4,1,1,0,0,1,20,26
...,...,...,...,...,...,...,...
10705,1996,1,0,0,0,2,3
10706,1996,1,0,0,0,2,4
10707,1996,0,0,0,0,2,3
10708,1996,0,0,0,0,3,7


In [80]:
indexes = []
pr_score = []
cr_score = []
rr_score = []
gr_score = []

for i in range(1997):
    indexes.append(i)
    pr_score.append(f22[f22['reviews']==i]['PS'].sum())
    cr_score.append(f22[f22['reviews']==i]['CS'].sum())
    rr_score.append(f22[f22['reviews']==i]['RS'].sum())
    gr_score.append(f22[f22['reviews']==i]['GS'].sum())

In [81]:
f10 = pd.DataFrame()
f10['reviews'] = indexes
f10['p_score'] = pr_score
f10['c_score'] = cr_score
f10['r_score'] = rr_score
f10['g_score'] = gr_score

In [82]:
f10

,reviews,p_score,c_score,r_score,g_score
0,0,1,0,0,1
1,1,3,0,2,3
2,2,3,0,0,0
3,3,4,0,1,3
4,4,-1,0,0,-1
...,...,...,...,...,...
1992,1992,0,0,0,1
1993,1993,-1,0,0,0
1994,1994,0,0,0,1
1995,1995,1,0,0,2


In [83]:
f10['brands'] = df['brand']

In [84]:
hpscore = f10[f10['brands']=='HP']
lescore = f10[f10['brands']=='LENOVO']
descore = f10[f10['brands']=='DELL']
mascore = f10[f10['brands']=='MAC']

In [85]:
print("proportion of negative performance reviews for LENOVO")
print()
print(np.round(len(lescore[lescore['p_score']<0])/(len(lescore[lescore['p_score']<0])+len(lescore[lescore['p_score']>0])), 2))
print()
print("proportion of negative performance reviews for HP")
print()
print(np.round(len(hpscore[hpscore['p_score']<0])/(len(hpscore[hpscore['p_score']<0])+len(hpscore[hpscore['p_score']>0])), 2))
print()
print("proportion of negative performance reviews for DELL")
print()
print(np.round(len(descore[descore['p_score']<0])/(len(descore[descore['p_score']<0])+len(descore[descore['p_score']>0])), 2))
print()
print("proportion of negative performance reviews for MAC")
print()
print(np.round(len(mascore[mascore['p_score']<0])/(len(mascore[mascore['p_score']<0])+len(mascore[mascore['p_score']>0])), 2))

proportion of negative performance reviews for LENOVO

0.61

proportion of negative performance reviews for HP

0.38

proportion of negative performance reviews for DELL

0.31

proportion of negative performance reviews for MAC

0.26


In [86]:
print("proportion of negative customer reviews for LENOVO")
print()
print(np.round(len(lescore[lescore['c_score']<0])/(len(lescore[lescore['c_score']<0])+len(lescore[lescore['c_score']>0])), 2))
print()
print("proportion of negative customer reviews for HP")
print()
print(np.round(len(hpscore[hpscore['c_score']<0])/(len(hpscore[hpscore['c_score']<0])+len(hpscore[hpscore['c_score']>0])), 2))
print()
print("proportion of negative customer reviews for DELL")
print()
print(np.round(len(descore[descore['c_score']<0])/(len(descore[descore['c_score']<0])+len(descore[descore['c_score']>0])), 2))
print()
print("proportion of negative customer reviews for MAC")
print()
print(np.round(len(mascore[mascore['c_score']<0])/(len(mascore[mascore['c_score']<0])+len(mascore[mascore['c_score']>0])), 2))

proportion of negative customer reviews for LENOVO

0.68

proportion of negative customer reviews for HP

0.48

proportion of negative customer reviews for DELL

0.49

proportion of negative customer reviews for MAC

0.48


In [87]:
print("proportion of negative price reviews for LENOVO")
print()
print(np.round(len(lescore[lescore['r_score']<0])/(len(lescore[lescore['r_score']<0])+len(lescore[lescore['r_score']>0])), 2))
print()
print("proportion of negative price reviews for HP")
print()
print(np.round(len(hpscore[hpscore['r_score']<0])/(len(hpscore[hpscore['r_score']<0])+len(hpscore[hpscore['r_score']>0])), 2))
print()
print("proportion of negative price reviews for DELL")
print()
print(np.round(len(descore[descore['r_score']<0])/(len(descore[descore['r_score']<0])+len(descore[descore['r_score']>0])), 2))
print()
print("proportion of negative price reviews for MAC")
print()
print(np.round(len(mascore[mascore['r_score']<0])/(len(mascore[mascore['r_score']<0])+len(mascore[mascore['r_score']>0])), 2))

proportion of negative price reviews for LENOVO

0.27

proportion of negative price reviews for HP

0.11

proportion of negative price reviews for DELL

0.21

proportion of negative price reviews for MAC

0.2


In [88]:
print("proportion of negative general reviews for LENOVO")
print()
print(np.round(len(lescore[lescore['g_score']<0])/(len(lescore[lescore['g_score']<0])+len(lescore[lescore['g_score']>0])), 2))
print()
print("proportion of negative general reviews for HP")
print()
print(np.round(len(hpscore[hpscore['g_score']<0])/(len(hpscore[hpscore['g_score']<0])+len(hpscore[hpscore['g_score']>0])), 2))
print()
print("proportion of negative general reviews for DELL")
print()
print(np.round(len(descore[descore['g_score']<0])/(len(descore[descore['g_score']<0])+len(descore[descore['g_score']>0])), 2))
print()
print("proportion of negative general reviews for MAC")
print()
print(np.round(len(mascore[mascore['g_score']<0])/(len(mascore[mascore['g_score']<0])+len(mascore[mascore['g_score']>0])), 2))

proportion of negative general reviews for LENOVO

0.42

proportion of negative general reviews for HP

0.21

proportion of negative general reviews for DELL

0.24

proportion of negative general reviews for MAC

0.27


In [114]:
indices = list(np.random.randint(0, 1997, size=210))

In [120]:
IN = list(set(indices))

In [124]:
finalin = IN[:-1]

In [126]:
valid = df.iloc[finalin]

In [128]:
valid.to_csv('/Users/Akashgupta/Desktop/data_files/HUMANCODERS.csv')

In [ ]:
pv_score = []
cv_score = []
rv_score = []
gv_score = []

for i in indices:
    print(i)
    print()
    print(valid['review'][i])
    print()
    p = input("performance score: ")
    c = input("customer score: ")
    r = input("price score: ")
    g = input("general score: ")
    print()
    print("======")
    pv_score.append(p)
    cv_score.append(c)
    rv_score.append(r)
    gv_score.append(g)

In [87]:
INDEX = pd.DataFrame()
INDEX['in'] = indices
INDEX.to_csv('/Users/Akashgupta/Desktop/data_files/indices.csv')

In [88]:
valid2 = valid.copy()

In [89]:
valid2['ps'] = pv_score
valid2['cs'] = cv_score
valid2['rs'] = rv_score
valid2['gs'] = gv_score

In [90]:
validation2 = valid2.copy()

In [91]:
validation3 = validation2.reset_index()

In [100]:
f12 = f10.iloc[indices]

In [101]:
f122 = f12.reset_index()

In [102]:
f12new = f122.copy()

In [ ]:
for i in range(len(f122['reviews'])):
    if f122['p_score'][i] < 0:
        f12new['p_score'][i] = -1
    elif f122['p_score'][i] == 0:
        f12new['p_score'][i] = 0
    elif f122['p_score'][i] > 0:
        f12new['p_score'][i] = 1
    
for i in range(len(f122['reviews'])):
    if f122['c_score'][i] < 0:
        f12new['c_score'][i] = -1
    elif f122['c_score'][i] == 0:
        f12new['c_score'][i] = 0
    elif f122['c_score'][i] > 0:
        f12new['c_score'][i] = 1
        
for i in range(len(f122['reviews'])):
    if f122['r_score'][i] < 0:
        f12new['r_score'][i] = -1
    elif f122['r_score'][i] == 0:
        f12new['r_score'][i] = 0
    elif f122['r_score'][i] > 0:
        f12new['r_score'][i] = 1
        
for i in range(len(f122['reviews'])):        
    if f122['g_score'][i] < 0:
        f12new['g_score'][i] = -1
    elif f122['g_score'][i] == 0:
        f12new['g_score'][i] = 0
    elif f122['g_score'][i] > 0:
        f12new['g_score'][i] = 1

In [106]:
validation3['ps'] = validation3['ps'].astype(int)
validation3['cs'] = validation3['cs'].astype(int)
validation3['rs'] = validation3['rs'].astype(int)
validation3['gs'] = validation3['gs'].astype(int)

In [107]:
pacc = 0
cacc = 0
racc = 0
gacc = 0

for i in range(len(f12new['reviews'])):
    if f12new['p_score'][i] == validation3['ps'][i]:
        pacc += 1

for i in range(len(f12new['reviews'])):
    if f12new['c_score'][i] == validation3['cs'][i]:
        cacc += 1
        
for i in range(len(f12new['reviews'])):
    if f12new['r_score'][i] == validation3['rs'][i]:
        racc += 1
        
for i in range(len(f12new['reviews'])):
    if f12new['g_score'][i] == validation3['gs'][i]:
        gacc += 1
        
PP = pacc/len(validation3['review'])
CC = cacc/len(validation3['review'])
RR = racc/len(validation3['review'])
GG = gacc/len(validation3['review'])

In [112]:
print('performance: ', PP)
print('customer: ', CC)
print('pricing: ', RR)
print('general:', GG)

performance:  0.7
customer:  0.88
pricing:  0.89
general: 0.65
